In [148]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,dataset
from torch.utils.data import sampler,TensorDataset
import torchvision.datasets as dset
import torchvision.transforms as T
from torchvision import transforms
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import cv2
import sys
import torchvision
dtype = torch.float32
print("CUDA: ",torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.ion()

def imshow(axis, inp):
    """Denormalize and show"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    axis.imshow(inp)

CUDA:  True
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [149]:
root = "/home/cth/code/cat_and_dog/data/train"

class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset.
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start=0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples


class Date(dataset.Dataset):
    def __init__(self,train=True,transform_train=None,transform_val=None):
        super(Date,self).__init__()
        self.train = train
        self.dir = os.listdir(root)
        self.len = len(self.dir)
        self.transform_train = transform_train
        self.transform_val = transform_val
    def __getitem__(self,index):
        path = self.dir[index]
        label = 0 if path[:3]=='cat' else 1
        img = cv2.imread(root+"/"+path)
        if self.train:
            img = self.transform_train(img)
        else:
            img = self.transform_val(img)
        return [img,label]
    def __len__(self):
        return self.len

batch_size = 64

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(224,scale=(0.08,1.0),ratio=(3.0/4.0,4.0/3.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4,contrast=0.4,saturation=0.4),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]
)

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.CenterCrop(224),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_train = Date(train=True,transform_train=transform_train,transform_val=transform_val)
data_val = Date(train=False,transform_train=transform_train,transform_val=transform_val)

loader_train = DataLoader(data_train,batch_size=batch_size,sampler=ChunkSampler(23000,0))
loader_val = DataLoader(data_val,batch_size=batch_size,sampler=ChunkSampler(1800,2000))

print(loader_train.__iter__().next()[0][0].shape)
print(data_train.len)

torch.Size([3, 224, 224])
25000


In [150]:
from torchvision.models import resnet50
resnet = resnet50(pretrained=True,progress=True)
for para in resnet.parameters():
    para.grad_require = False
in_channel = resnet.fc.in_features
resnet.fc = torch.nn.Linear(in_channel,1024)

model = torch.nn.Sequential(
    resnet,
    torch.nn.ReLU(),
    torch.nn.Linear(1024,2)
)
# data = loader_train.__iter__().next()[0][0].unsqueeze(0)
# print(model(data).shape)

In [151]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return acc

def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    global bestacc,bestmodel
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            loss = F.cross_entropy(scores, y)
            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()
            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()
            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                acc = check_accuracy_part34(loader_val, model)
                #check_accuracy_part34(loader_train, model)
                if acc > bestacc:
                    bestacc = acc
                    bestmodel = copy.deepcopy(model)
                print()
                sys.stdout.flush()
        torch.save(model,"./model_save/"+str(e))

In [153]:
# import copy
# bestacc = 0
# bestmodel = model
# print_every = 100
# optimizer = torch.optim.SGD((para for para in model.parameters() if para.requires_grad),lr=1e-3,momentum=0.9)
# train_part34(model, optimizer , 10)
# torch.save(bestmodel,"./1bestmodel")

check_accuracy_part34(loader_train, model)

Checking accuracy on validation set
Got 22540 / 23000 correct (98.00)


0.98

In [161]:
#0 猫 1 狗
from pandas import DataFrame
testroot = "/home/cth/code/cat_and_dog/data/test1"
listdir = list(sorted(os.listdir(testroot),key=lambda x : int(x[:-4])))
model.eval()
data = []
model = torch.load("/home/cth/code/cat_and_dog/1bestmodel")
for idx,path in enumerate(listdir):
    img = cv2.imread(testroot+"/"+path)
    img = transform_val(img).unsqueeze(0)
    img = img.to(device)
    id = int(path[:-4])
    score = model(img)
    score = F.softmax(score,dim=1).detach().cpu().numpy()
    data.append((id,score[0][1]))
    if idx % 100 == 0:
        print(idx)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400


In [157]:
df = DataFrame(data,columns=["id","label"])
df.to_csv("submission.csv")

In [159]:
d = pd.read_csv("submission.csv",index_col="id")
d = d.iloc[:,1:]
# d.to_csv("sub.csv")
d
d.to_csv("res.csv")

In [116]:
data

tensor([[-2.9031,  3.0385]], device='cuda:0', grad_fn=<CatBackward>)

In [123]:
F.softmax(data,dim=1).detach().cpu().numpy()

array([[0.00262111, 0.99737895]], dtype=float32)